# Semantic Kernel OpenAI Assistant Agent Code Interpreter

In [ ]:
%pip install --upgrade semantic-kernel==1.20.0

## Azure Resources Needed
1. Azure OpenAI
    - Deploy GPT-4o


## Prepare File Paths of our files to be added to agent

In [1]:
import os

# Let's form the file paths that we will later pass to the assistant
csv_file_path_1 = os.path.join("Data/population/","PopulationByAdmin1.csv",)

csv_file_path_2 = os.path.join("Data/population/","PopulationByCountry.csv",)

## Step 1-2: Create an Agent and Thread

In [15]:
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.streaming_file_reference_content import StreamingFileReferenceContent
from semantic_kernel.contents.file_reference_content import FileReferenceContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

# Step 1: Create an assistant agent
agent = await AzureAssistantAgent.create(
        kernel=Kernel(),
        service_id="agent",
        name="SKCodeInterpreter",
        instructions="""
                Analyze the available data to provide an answer to the user's question.
                Always format response using markdown.
                Always include a numerical index that starts at 1 for any lists or tables.
                Always sort lists in ascending order.
                """,
        enable_code_interpreter=True,
        code_interpreter_filenames=[csv_file_path_1, csv_file_path_2],
)

# Step 2: Create a thread
thread_id = await agent.create_thread()

## Helper Methods
This will download any files created by the agent

In [16]:
async def download_file_content(agent, file_id: str):
    try:
        # Fetch the content of the file using the provided method
        response_content = await agent.client.files.content(file_id)

        # Get the current working directory of the file
        current_directory = "Data/skagents-output/"

        # Define the path to save the image in the current directory
        file_path = os.path.join(
            current_directory,  # Use the current directory of the file
            f"{file_id}.png",  # You can modify this to use the actual filename with proper extension
        )

        # Save content to a file asynchronously
        with open(file_path, "wb") as file:
            file.write(response_content.content)

        print(f"File saved to: {file_path}")
    except Exception as e:
        print(f"An error occurred while downloading file {file_id}: {str(e)}")


async def download_response_image(agent, file_ids: list[str]):
    if file_ids:
        # Iterate over file_ids and download each one
        for file_id in file_ids:
            await download_file_content(agent, file_id)

## Step 3-5: Helper Function
3. Add a message to the thread
4. Run the Assistant
5. Display the Assistant's Response

In [17]:
async def run_agent(user_question):
    # File IDs to store the references to the files
    file_ids: list[str] = []

    # STEP 3: Add a user question to the thread
    await agent.add_chat_message(
        thread_id=thread_id, 
        message=ChatMessageContent(role=AuthorRole.USER, content=user_question)
    )

    # STEP 4: Invoke the agent to get a response
    async for response in agent.invoke(thread_id=thread_id):
        # check if the response contains any code
        if response.metadata.get("code"):
            print("\n" + "Code from Agent:")
            #STEP 5: Print the Assistant response
            print(f"{response.content}", end="", flush=True)        
        else:
            #STEP 5: Print the Assistant response
            print(f"{response.content}", end="", flush=True)
                    
        print("\n**************")
        #Check if the response contains any file references
        file_ids.extend([item.file_id for item in response.items if isinstance(item, FileReferenceContent)])

    # Download the file content
    await download_response_image(agent, file_ids)
    file_ids.clear()


In [18]:
user_question = "What is the population of the country with the highest population?"
await run_agent(user_question)


Code from Agent:
import pandas as pd

# Load the contents of the uploaded files to examine
file1_path = '/mnt/data/assistant-QBYhVsUviV2iMYM5e8RVe8'
file2_path = '/mnt/data/assistant-BHeS4FM3DPeSm6QZ4Qzbxv'

file1 = pd.read_excel(file1_path)
file2 = pd.read_excel(file2_path)

file1.head(), file2.head()
**************

Code from Agent:
# Let's try reading the files with the 'openpyxl' engine for .xlsx files
file1 = pd.read_excel(file1_path, engine='openpyxl')
file2 = pd.read_excel(file2_path, engine='openpyxl')

file1.head(), file2.head()
**************

Code from Agent:
import magic

# Function to determine file type using magic library
def determine_file_type(file_path):
    file_magic = magic.Magic(mime=True)
    file_type = file_magic.from_file(file_path)
    return file_type

# Determine the file types for both files
file1_type = determine_file_type(file1_path)
file2_type = determine_file_type(file2_path)

file1_type, file2_type
**************

Code from Agent:
file1_csv = pd.read

## Appending Messages to the Thread

In [19]:
user_question = "What is in third place?"
await run_agent(user_question)


Code from Agent:
# Finding the country with the third highest population
sorted_population = file2_csv.sort_values(by="Population", ascending=False)
third_highest_population = sorted_population.iloc[2]
third_highest_population
**************
The country with the third highest population is **the United States** with a population of **329,466,283**.
**************


In [20]:
user_question = "Provide a column chart of the top 10 countries by population. Add values at the top of each column"
await run_agent(user_question)

AgentInvokeException: Run failed with status: `failed` for agent `SKCodeInterpreter` and thread `thread_9XlmSEWZJ8NZMxftcsQnLf5K` with error: Sorry, something went wrong.

In [ ]:
user_question = "Provide a column chart for countries whose names start with the same letter and sort the x axis by highest count to lowest (include all countries). Add values on top of each column"
await run_agent(user_question)

## Deleting Files, Thread, Agent

In [ ]:
if agent is not None:
    [await agent.delete_file(file_id) for file_id in agent.code_interpreter_file_ids]
    await agent.delete_thread(thread_id)
    await agent.delete()